In [1]:
#import modules
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

pd.options.display.max_columns=999
pd.options.display.max_rows=3000

In [2]:

from openpyxl.workbook import Workbook


In [114]:
df = pd.read_excel('Collections Case Study Data.xlsx', sheet_name=None)

KeyboardInterrupt: 

## What is the account roll forward rate for the months of Mar , Apr, May and June for TB0?

In [95]:
start_data = df['Loan Details']
mar1_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'March']
apr1_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'April']
may1_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'May']
apr2_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'April']
may2_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'May']
jun2_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'June']

In [96]:
start_data.head()

,Loan_id,Debt_to_burden_Ratio,total_income,TENURE,Sanctioned Amount
0,A999993387,0.259113,5326806.0,36,22000000
1,A999990339,0.323385,8392773.0,48,73000000
2,A999971854,0.346267,12000000.0,48,80000000
3,A999952066,0.229800,9000000.0,36,50000000
4,A999938479,0.370082,8500000.0,36,67000000


In [97]:
dfs = [start_data, mar1_data, apr1_data, may1_data, apr2_data, may2_data, jun2_data]

In [99]:
data = start_data

In [100]:
for df in dfs:
    data1 = pd.merge(data, df, how='left', on='Loan_id')

In [101]:
data1.head()

,Loan_id,Debt_to_burden_Ratio,total_income,TENURE_x,Sanctioned Amount_x,Bucket,PRINBALANCE,TENURE_y,Months on Books,Sanctioned Amount_y,Month,Unnamed: 7,Unnamed: 8
0,A999993387,0.259113,5326806.0,36,22000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A999990339,0.323385,8392773.0,48,73000000,REGULAR,66303700.0,48.0,7.0,73000000.0,June,NaN,NaN
2,A999971854,0.346267,12000000.0,48,80000000,REGULAR,48509900.0,48.0,25.0,80000000.0,June,NaN,-19605000.0
3,A999952066,0.229800,9000000.0,36,50000000,REGULAR,32791800.0,36.0,15.0,50000000.0,June,NaN,NaN
4,A999938479,0.370082,8500000.0,36,67000000,REGULAR,60348600.0,36.0,5.0,67000000.0,June,NaN,NaN


In [102]:
from functools import reduce
dfs = [start_data, mar1_data, apr1_data, may1_data, apr2_data, may2_data, jun2_data]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Loan_id'],how='left'), dfs)

In [103]:
df_merged.head()

,Loan_id,Debt_to_burden_Ratio,total_income,TENURE_x,Sanctioned Amount_x,Bucket_x,TENURE_y,PRINBALANCE_x,Months on Books_x,Sanctioned Amount_y,Month_x,Unnamed: 7_x,Unnamed: 8_x,Bucket_y,TENURE_x,PRINBALANCE_y,Months on Books_y,Sanctioned Amount_x,Month_y,Unnamed: 7_y,Unnamed: 8_y,Bucket_x,TENURE_y,PRINBALANCE_x,Months on Books_x,Sanctioned Amount_y,Month_x,Unnamed: 7_x,Unnamed: 8_x,Bucket_y,PRINBALANCE_y,TENURE_x,Months on Books_y,Sanctioned Amount_x,Month_y,Unnamed: 7_y,Unnamed: 8_y,Bucket_x,PRINBALANCE_x,TENURE_y,Months on Books_x,Sanctioned Amount_y,Month_x,Unnamed: 7_x,Unnamed: 8_x,Bucket_y,PRINBALANCE_y,TENURE,Months on Books_y,Sanctioned Amount,Month_y,Unnamed: 7_y,Unnamed: 8_y
0,A999993387,0.259113,5326806.0,36,22000000,TB0,36.0,14361500.0,17.0,22000000.0,March,NaN,576300.0,TB0,36.0,13785200.0,18.0,22000000.0,April,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REGULAR,13192100.0,36.0,18.0,22000000.0,April,NaN,609100.0,REGULAR,12583000.0,36.0,19.0,22000000.0,May,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A999990339,0.323385,8392773.0,48,73000000,TB0,48.0,69805500.0,4.0,73000000.0,March,NaN,848000.0,TB0,48.0,68957500.0,5.0,73000000.0,April,NaN,842600.0,TB0,48.0,68114900.0,6.0,73000000.0,May,NaN,NaN,REGULAR,68114900.0,48.0,5.0,73000000.0,April,NaN,NaN,REGULAR,67206100.0,48.0,6.0,73000000.0,May,NaN,NaN,REGULAR,66303700.0,48.0,7.0,73000000.0,June,NaN,NaN
2,A999971854,0.346267,12000000.0,48,80000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TB0,48.0,53248600.0,23.0,80000000.0,April,NaN,1550600.0,TB0,48.0,51698000.0,24.0,80000000.0,May,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REGULAR,50123600.0,48.0,24.0,80000000.0,May,NaN,1613700.0,REGULAR,48509900.0,48.0,25.0,80000000.0,June,NaN,-19605000.0
3,A999952066,0.229800,9000000.0,36,50000000,TB0,36.0,37728800.0,12.0,50000000.0,March,NaN,1237600.0,TB0,36.0,36491200.0,13.0,50000000.0,April,NaN,1216700.0,TB0,36.0,35274500.0,14.0,50000000.0,May,NaN,NaN,REGULAR,35274500.0,36.0,13.0,50000000.0,April,NaN,NaN,REGULAR,34047600.0,36.0,14.0,50000000.0,May,NaN,NaN,REGULAR,32791800.0,36.0,15.0,50000000.0,June,NaN,NaN
4,A999938479,0.370082,8500000.0,36,67000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TB0,36.0,63848300.0,3.0,67000000.0,April,NaN,NaN,TB0,36.0,62723900.0,4.0,67000000.0,May,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REGULAR,61587400.0,36.0,4.0,67000000.0,May,NaN,NaN,REGULAR,60348600.0,36.0,5.0,67000000.0,June,NaN,NaN


In [104]:
df_merged.columns

Index(['Loan_id', 'Debt_to_burden_Ratio', 'total_income', 'TENURE_x',
       'Sanctioned Amount_x', 'Bucket_x', 'TENURE_y', 'PRINBALANCE_x',
       'Months on Books_x', 'Sanctioned Amount_y', 'Month_x', 'Unnamed: 7_x',
       'Unnamed: 8_x', 'Bucket_y', 'TENURE_x', 'PRINBALANCE_y',
       'Months on Books_y', 'Sanctioned Amount_x', 'Month_y', 'Unnamed: 7_y',
       'Unnamed: 8_y', 'Bucket_x', 'TENURE_y', 'PRINBALANCE_x',
       'Months on Books_x', 'Sanctioned Amount_y', 'Month_x', 'Unnamed: 7_x',
       'Unnamed: 8_x', 'Bucket_y', 'PRINBALANCE_y', 'TENURE_x',
       'Months on Books_y', 'Sanctioned Amount_x', 'Month_y', 'Unnamed: 7_y',
       'Unnamed: 8_y', 'Bucket_x', 'PRINBALANCE_x', 'TENURE_y',
       'Months on Books_x', 'Sanctioned Amount_y', 'Month_x', 'Unnamed: 7_x',
       'Unnamed: 8_x', 'Bucket_y', 'PRINBALANCE_y', 'TENURE',
       'Months on Books_y', 'Sanctioned Amount', 'Month_y', 'Unnamed: 7_y',
       'Unnamed: 8_y'],
      dtype='object')

In [105]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40339 entries, 0 to 40338
Data columns (total 53 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Loan_id               40339 non-null  object 
 1   Debt_to_burden_Ratio  40339 non-null  float64
 2   total_income          40339 non-null  float64
 3   TENURE_x              40339 non-null  int64  
 4   Sanctioned Amount_x   40339 non-null  int64  
 5   Bucket_x              24575 non-null  object 
 6   TENURE_y              24575 non-null  float64
 7   PRINBALANCE_x         24575 non-null  float64
 8   Months on Books_x     24575 non-null  float64
 9   Sanctioned Amount_y   24575 non-null  float64
 10  Month_x               24575 non-null  object 
 11  Unnamed: 7_x          0 non-null      float64
 12  Unnamed: 8_x          3 non-null      float64
 13  Bucket_y              27079 non-null  object 
 14  TENURE_x              27079 non-null  float64
 15  PRINBALANCE_y      

In [106]:
df2 = df_merged[['Loan_id', 'Bucket_x', 'Bucket_y']]

In [107]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40339 entries, 0 to 40338
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Loan_id   40339 non-null  object
 1   Bucket_x  24575 non-null  object
 2   Bucket_x  29115 non-null  object
 3   Bucket_x  27079 non-null  object
 4   Bucket_y  27079 non-null  object
 5   Bucket_y  24575 non-null  object
 6   Bucket_y  29115 non-null  object
dtypes: object(7)
memory usage: 2.5+ MB


In [108]:
df2.columns = ['Loan_id', 'Bucket_mar1', 'Bucket_may1', 'Bucket_may2', 'Bucket_apr1', 'Bucket_apr2', 'Bucket_jun2']
df2 = df2[['Loan_id', 'Bucket_mar1', 'Bucket_apr1', 'Bucket_may1', 'Bucket_apr2', 'Bucket_may2', 'Bucket_jun2']]
df2.head()

,Loan_id,Bucket_mar1,Bucket_apr1,Bucket_may1,Bucket_apr2,Bucket_may2,Bucket_jun2
0,A999993387,TB0,TB0,NaN,REGULAR,REGULAR,NaN
1,A999990339,TB0,TB0,TB0,REGULAR,REGULAR,REGULAR
2,A999971854,NaN,TB0,TB0,NaN,REGULAR,REGULAR
3,A999952066,TB0,TB0,TB0,REGULAR,REGULAR,REGULAR
4,A999938479,NaN,TB0,TB0,NaN,REGULAR,REGULAR


In [110]:
cols = df2.columns

for c in cols:
    print(df2[c].value_counts())

A277860021    1
A737756929    1
A409493772    1
A644241821    1
A656707551    1
             ..
A7152365      1
A642857773    1
A10699572     1
A891798984    1
A984001610    1
Name: Loan_id, Length: 40339, dtype: int64
TB0    24575
Name: Bucket_mar1, dtype: int64
TB0    27079
Name: Bucket_apr1, dtype: int64
TB0    29115
Name: Bucket_may1, dtype: int64
REGULAR    22475
TB0         1709
TB1          391
Name: Bucket_apr2, dtype: int64
REGULAR    24964
TB0         1574
TB1          541
Name: Bucket_may2, dtype: int64
REGULAR    27085
TB0         1577
TB1          453
Name: Bucket_jun2, dtype: int64


In [ ]:
def clean_bucket(row):
    if row[]

In [71]:
df2['start_bucket'] = 'REGULAR'
df2.head()

,Loan_id,Bucket_x,Bucket_x,Bucket_y,Bucket_y,start_bucket
0,A999993387,TB0,REGULAR,REGULAR,NaN,REGULAR
1,A999990339,TB0,REGULAR,REGULAR,REGULAR,REGULAR
2,A999971854,NaN,REGULAR,NaN,REGULAR,REGULAR
3,A999952066,TB0,REGULAR,REGULAR,REGULAR,REGULAR
4,A999938479,NaN,REGULAR,NaN,REGULAR,REGULAR


In [72]:
df2.head()

,Loan_id,Bucket_x,Bucket_x,Bucket_y,Bucket_y,start_bucket
0,A999993387,TB0,REGULAR,REGULAR,NaN,REGULAR
1,A999990339,TB0,REGULAR,REGULAR,REGULAR,REGULAR
2,A999971854,NaN,REGULAR,NaN,REGULAR,REGULAR
3,A999952066,TB0,REGULAR,REGULAR,REGULAR,REGULAR
4,A999938479,NaN,REGULAR,NaN,REGULAR,REGULAR


In [74]:
cols = df2.columns

for c in cols:
    print(df2[c].value_counts())

A277860021    1
A737756929    1
A409493772    1
A644241821    1
A656707551    1
             ..
A7152365      1
A642857773    1
A10699572     1
A891798984    1
A984001610    1
Name: Loan_id, Length: 40339, dtype: int64
REGULAR    40339
Name: Bucket_start, dtype: int64
TB0    24575
Name: Bucket_mar, dtype: int64
REGULAR    22475
TB0         1709
TB1          391
Name: Bucket_apr, dtype: int64
REGULAR    24964
TB0         1574
TB1          541
Name: Bucket_may, dtype: int64
REGULAR    27085
TB0         1577
TB1          453
Name: Bucket_jun, dtype: int64


In [75]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40339 entries, 0 to 40338
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Loan_id       40339 non-null  object
 1   Bucket_start  40339 non-null  object
 2   Bucket_mar    24575 non-null  object
 3   Bucket_apr    24575 non-null  object
 4   Bucket_may    27079 non-null  object
 5   Bucket_jun    29115 non-null  object
dtypes: object(6)
memory usage: 3.4+ MB


In [76]:
df2[df2['Bucket_mar'].isnull()]

,Loan_id,Bucket_start,Bucket_mar,Bucket_apr,Bucket_may,Bucket_jun
2,A999971854,REGULAR,NaN,NaN,REGULAR,REGULAR
4,A999938479,REGULAR,NaN,NaN,REGULAR,REGULAR
6,A999913454,REGULAR,NaN,NaN,REGULAR,REGULAR
7,A999891961,REGULAR,NaN,NaN,REGULAR,REGULAR
9,A999870375,REGULAR,NaN,NaN,NaN,REGULAR
...,...,...,...,...,...,...
40326,A100462080,REGULAR,NaN,NaN,REGULAR,NaN
40329,A100223932,REGULAR,NaN,NaN,REGULAR,REGULAR
40330,A100220970,REGULAR,NaN,NaN,NaN,REGULAR
40331,A100184245,REGULAR,NaN,NaN,NaN,REGULAR


In [77]:
df2 = df2.fillna('REGULAR')

In [78]:
df2.head()

,Loan_id,Bucket_start,Bucket_mar,Bucket_apr,Bucket_may,Bucket_jun
0,A999993387,REGULAR,TB0,REGULAR,REGULAR,REGULAR
1,A999990339,REGULAR,TB0,REGULAR,REGULAR,REGULAR
2,A999971854,REGULAR,REGULAR,REGULAR,REGULAR,REGULAR
3,A999952066,REGULAR,TB0,REGULAR,REGULAR,REGULAR
4,A999938479,REGULAR,REGULAR,REGULAR,REGULAR,REGULAR


In [79]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40339 entries, 0 to 40338
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Loan_id       40339 non-null  object
 1   Bucket_start  40339 non-null  object
 2   Bucket_mar    40339 non-null  object
 3   Bucket_apr    40339 non-null  object
 4   Bucket_may    40339 non-null  object
 5   Bucket_jun    40339 non-null  object
dtypes: object(6)
memory usage: 3.4+ MB


In [80]:
cols = df2.columns

for c in cols:
    print(df2[c].value_counts())

A277860021    1
A737756929    1
A409493772    1
A644241821    1
A656707551    1
             ..
A7152365      1
A642857773    1
A10699572     1
A891798984    1
A984001610    1
Name: Loan_id, Length: 40339, dtype: int64
REGULAR    40339
Name: Bucket_start, dtype: int64
TB0        24575
REGULAR    15764
Name: Bucket_mar, dtype: int64
REGULAR    38239
TB0         1709
TB1          391
Name: Bucket_apr, dtype: int64
REGULAR    38224
TB0         1574
TB1          541
Name: Bucket_may, dtype: int64
REGULAR    38309
TB0         1577
TB1          453
Name: Bucket_jun, dtype: int64


In [86]:
bucket_counts = df2.agg({'Bucket_start': 'value_counts', 
                        'Bucket_mar': 'value_counts',
                        'Bucket_apr': 'value_counts',
                        'Bucket_may': 'value_counts',
                        'Bucket_jun': 'value_counts'})

bucket_counts

,Bucket_start,Bucket_mar,Bucket_apr,Bucket_may,Bucket_jun
REGULAR,40339.0,15764.0,38239,38224,38309
TB0,NaN,24575.0,1709,1574,1577
TB1,NaN,NaN,391,541,453


In [89]:
bucket_counts.iloc[1,1]

24575.0

In [94]:
mar_roll_rate = (bucket_counts.iloc[1,1] / bucket_counts.iloc[0,0]) *100
print('March Roll Rate: '+ str(mar_roll_rate) + '%')

March Roll Rate: 60.92119289025509%


## What is the average attempt intensity, RPC rate and PTP rate on accounts for the months of mar, Apr and May

In [113]:
df.keys()

Index(['Loan_id', 'Bucket', 'PRINBALANCE', 'TENURE', 'Months on Books',
       'Sanctioned Amount', 'Month', 'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')

In [111]:
call_data = df['Call Details']
call_data.head()

KeyError: 'Call Details'